In [1]:
import numpy
import urllib
import scipy.optimize
import random
from collections import defaultdict
import nltk
import string
from nltk.stem.porter import *
from sklearn import linear_model
import xgboost as xgb
from sklearn.metrics import accuracy_score ,confusion_matrix
from sklearn.model_selection import train_test_split  

In [2]:
for l in open('News_Category_Dataset.json'):
    print (l)
    break

{"short_description": "She left her husband. He killed their children. Just another day in America.", "headline": "There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV", "date": "2018-05-26", "link": "https://www.huffingtonpost.com/entry/texas-amanda-painter-mass-shooting_us_5b081ab4e4b0802d69caad89", "authors": "Melissa Jeltsen", "category": "CRIME"}



In [3]:
def readjson(f):
  for l in open(f):
    yield eval(l)

In [4]:
data=readjson('News_Category_Dataset.json')
data=list(data)
import nltk
from nltk.corpus import stopwords
sw = set(stopwords.words('english'))
print (sw)

set([u'all', u'just', u"don't", u'being', u'over', u'both', u'through', u'yourselves', u'its', u'before', u'o', u'don', u'hadn', u'herself', u'll', u'had', u'should', u'to', u'only', u'won', u'under', u'ours', u'has', u"should've", u"haven't", u'do', u'them', u'his', u'very', u"you've", u'they', u'not', u'during', u'now', u'him', u'nor', u"wasn't", u'd', u'did', u'didn', u'this', u'she', u'each', u'further', u"won't", u'where', u"mustn't", u"isn't", u'few', u'because', u"you'd", u'doing', u'some', u'hasn', u"hasn't", u'are', u'our', u'ourselves', u'out', u'what', u'for', u"needn't", u'below', u're', u'does', u"shouldn't", u'above', u'between', u'mustn', u't', u'be', u'we', u'who', u"mightn't", u"doesn't", u'were', u'here', u'shouldn', u'hers', u"aren't", u'by', u'on', u'about', u'couldn', u'of', u"wouldn't", u'against', u's', u'isn', u'or', u'own', u'into', u'yourself', u'down', u"hadn't", u'mightn', u"couldn't", u'wasn', u'your', u"you're", u'from', u'her', u'their', u'aren', u"it's",

In [5]:
w = [d['short_description']+" "+d['headline']+" "+d['authors'] for d in data]

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(analyzer = "word", max_features=5000, stop_words=sw)
X = tfidf.fit_transform(w).toarray()  

In [16]:
y = [d['category'] for d in data]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) 

#### 5000 features

In [33]:
from sklearn.svm import LinearSVC

model = LinearSVC()
model.fit(X_train,y_train)
y_predict = model.predict(X_test)
accuracy = accuracy_score(y_test,y_predict)*100
print(format(accuracy, '.2f'))

70.39


#### 3000 features

In [17]:
ylabel_id = dict(zip(list(set(y)), range(31)) )
dtrain=[ int(ylabel_id[d])  for d in y ]
X=numpy.array(X)
y=numpy.array(dtrain)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) 

In [19]:
xg_train = xgb.DMatrix(X_train, label=y_train)
xg_test = xgb.DMatrix(X_test, label=y_test)

In [20]:
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.1
param['max_depth'] = 6
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 31

In [21]:
watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 100
bst = xgb.train(param, xg_train, num_round, watchlist)
# get prediction
pred = bst.predict(xg_test)
error_rate = numpy.sum(pred != y_test) / (y_test.shape[0]+0.0)
print('Test error using softmax = {}'.format(error_rate))

[0]	train-merror:0.490164	test-merror:0.499200
[1]	train-merror:0.448880	test-merror:0.460437
[2]	train-merror:0.434859	test-merror:0.447676
[3]	train-merror:0.431749	test-merror:0.447876
[4]	train-merror:0.428269	test-merror:0.442275
[5]	train-merror:0.426148	test-merror:0.440715
[6]	train-merror:0.423458	test-merror:0.438235
[7]	train-merror:0.420848	test-merror:0.437515
[8]	train-merror:0.417898	test-merror:0.435115
[9]	train-merror:0.415177	test-merror:0.434435
[10]	train-merror:0.413327	test-merror:0.433115
[11]	train-merror:0.411367	test-merror:0.431675
[12]	train-merror:0.408567	test-merror:0.429474
[13]	train-merror:0.407257	test-merror:0.428354
[14]	train-merror:0.406037	test-merror:0.428514
[15]	train-merror:0.403026	test-merror:0.426074
[16]	train-merror:0.401076	test-merror:0.425234
[17]	train-merror:0.398366	test-merror:0.423114
[18]	train-merror:0.396206	test-merror:0.420674
[19]	train-merror:0.392565	test-merror:0.418714
[20]	train-merror:0.391175	test-merror:0.418313
[2

Test error using softmax = 0.359588767101
